# Análisis de Reviews de Apps

Este notebook analiza las reviews de apps extraídas de Google Play y App Store. Se realizan análisis de sentimiento, clasificación de reviews y visualización de resultados.

## Instalación de requerimientos



In [ ]:
# Python 3.9.6
!pip install google-play-scraper sentence-transformers matplotlib seaborn pandas feedparser transformers nltk wordcloud pysentimiento

## Importación de librerías

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud

# Importar todas las funciones y variables útiles
from helpers import (
    get_playstore_reviews,
    get_itunes_reviews,
    analyze_sentiment,
    classify_keywords_with_sentiment,
    extract_topics,
    classify_review_topic,
    preprocess_text,
    model,  # Global model for embeddings
    keyword_embeddings,  # Precomputed embeddings for keywords
    topic_seeds  # Seed words for bug topics
)

sns.set(style="whitegrid")

## Apps a analizar



In [ ]:
# Definir un diccionario con los IDs de cada app
BELO_IDS = {
    "play_store_id": "com.belo.android",
    "app_store_id": "1575614708"
}
ASTROPAY_IDS = {
    "play_store_id": "com.astropaycard.android",
    "app_store_id": "1128476912"
}

apps = {
    "belo": BELO_IDS,
    # "astropay": ASTROPAY_IDS
}

# Lista para almacenar los DataFrames de reviews
all_reviews_list = []

# Iterar sobre cada app y obtener reviews de ambas tiendas
for app_name, ids in apps.items():
    # Reviews desde Google Play
    play_reviews = get_playstore_reviews(ids["play_store_id"])
    if not play_reviews.empty:
        play_reviews["app"] = app_name
        play_reviews["store"] = "Play Store"
        all_reviews_list.append(play_reviews)
    
    # Reviews desde App Store (iTunes RSS)
    itunes_reviews = get_itunes_reviews(ids["app_store_id"])
    if not itunes_reviews.empty:
        itunes_reviews["app"] = app_name
        itunes_reviews["store"] = "App Store"
        all_reviews_list.append(itunes_reviews)

# Concatenar todas las reviews en un único DataFrame
all_reviews = pd.concat(all_reviews_list, ignore_index=True)

## Analisis de sentimiento

In [ ]:
# Análisis de sentimiento usando pysentimiento
all_reviews["sentiment"] = all_reviews["content"].apply(analyze_sentiment)

# Mostrar los primeros 10 reseñas
all_reviews.head(10)

### Visualizacion del analisis de sentimiento

In [ ]:
# Agrupar por app y sentimiento
sentiment_counts_app = all_reviews.groupby(['app', 'sentiment']).size().reset_index(name='Count')

plt.figure(figsize=(10, 6))
sns.barplot(x='sentiment', y='Count', hue='app', data=sentiment_counts_app, palette='viridis')
plt.title("Distribución de Sentimientos por App")
plt.xlabel("Sentimiento")
plt.ylabel("Cantidad de Reviews")
plt.show()

# Agrupar por semana, app y sentimiento
all_reviews['week'] = pd.to_datetime(all_reviews['date']).dt.to_period("W").apply(lambda r: r.start_time)
weekly_sentiment_app = all_reviews.groupby(['week', 'app', 'sentiment']).size().reset_index(name='Count')

plt.figure(figsize=(12, 6))
sns.lineplot(x='week', y='Count', hue='sentiment', style='app', data=weekly_sentiment_app, markers=True)
plt.title("Evolución Semanal de los Sentimientos por App")
plt.xlabel("Semana")
plt.ylabel("Cantidad de Reviews")
plt.xticks(rotation=45)
plt.legend(title="Sentimiento / App")
plt.show()

# Mapear el sentimiento a un score numérico
sentiment_map = {"POS": 1, "NEU": 0, "NEG": -1}
all_reviews["sentiment_score"] = all_reviews["sentiment"].map(sentiment_map)

# Visualización: Rating vs Sentiment Score
plt.figure(figsize=(10, 6))
# Boxplot para mostrar la distribución de 'sentiment_score' por rating
sns.boxplot(x="rating", y="sentiment_score", data=all_reviews, hue="app", showfliers=False)
# Stripplot para mostrar cada punto
sns.stripplot(x="rating", y="sentiment_score", data=all_reviews, hue="app", 
              dodge=True, alpha=0.5, color='black', jitter=True)

plt.title("Distribución de Sentiment Score por Rating (posible ironía)")
plt.xlabel("Rating")
plt.ylabel("Sentiment Score")
plt.legend(title="App", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.show()

## Extraccion de bugs y features

Extraccion de reporte de bugs y solicitudes de features

In [ ]:
# Clasificar cada review en bug/feature utilizando la función importada
tags = all_reviews.apply(classify_keywords_with_sentiment, axis=1)
all_reviews = pd.concat([all_reviews, tags], axis=1)

# Mostrar los primeros 10 reseñas con su clasificación
display(all_reviews[['content', 'sentiment', 'is_bug', 'is_feature']].head(10))


### Listados de features y bugs

In [ ]:
# Listado de Feature Requests
features_reviews = all_reviews[all_reviews['is_feature'] == True]
print("Listado de Feature Requests:")
display(features_reviews[['date', 'app', 'appVersion', 'store', 'content', 'sentiment']].sort_values(by='date', ascending=False).reset_index(drop=True))

# Listado de Bug Reports para la app 'belo'
bugs_reviews = all_reviews[(all_reviews['is_bug'] == True) & (all_reviews['app'] == 'belo')]
print("Listado de Bug Reports:")
display(bugs_reviews[['date', 'app', 'appVersion', 'store', 'content', 'sentiment']].sort_values(by='date', ascending=False).reset_index(drop=True))

## Topicos de las reviews

In [ ]:
# Asignar tópico a cada review
all_reviews["topic"] = all_reviews["content"].apply(classify_review_topic)

# Mostrar los primeros 10 reseñas con su tópico
display(all_reviews[['content', 'topic']].head(10))


### Visualizacion de los topicos
Distribucion de reviews por topico

In [ ]:
# Gráfico de barras: Distribución de reviews por topico
# Todos los topicos
# topic_counts = all_reviews["topic"].value_counts()

# Bug Reports
topic_counts = all_reviews[all_reviews["is_bug"] == True]["topic"].value_counts()

# Feature Requests
# topic_counts = all_reviews[all_reviews["is_feature"] == True]["topic"].value_counts()

plt.figure(figsize=(10, 6))
sns.barplot(x=topic_counts.index, y=topic_counts.values, palette="viridis")
plt.title("Distribución de Reviews por Tema")
plt.xlabel("Tema")
plt.ylabel("Cantidad de Reviews")
plt.xticks(rotation=45)
plt.show()

# Generar y visualizar un Word Cloud para cada tema
# for topic in topic_counts.index:
#     # Combina el contenido de las reviews del tema
#     text = " ".join(all_reviews[all_reviews["topic"] == topic]["content"].tolist())
    
#     # Generar la nube de palabras
#     wordcloud = WordCloud(width=800, height=400, background_color="white", colormap="viridis").generate(text)
    
#     # Visualizar la nube de palabras
#     plt.figure(figsize=(10, 5))
#     plt.imshow(wordcloud, interpolation="bilinear")
#     plt.axis("off")
#     plt.title(f"Word Cloud para el tema: {topic}")
#     plt.show()

## Visualizacion de reviews

Evolicion semanal del promedio de rating, cantidad de reviews, sentimiento positivo y sentimiento negativo

In [ ]:
# Asegurarse de que las columnas 'sentiment', 'is_bug' e 'is_feature' existan
for col in ['sentiment', 'is_bug', 'is_feature']:
    if col not in all_reviews.columns:
         all_reviews[col] = None  

weekly_summary = all_reviews.groupby(['app', 'store', 'week']).agg(
    avg_rating=('rating', 'mean'),
    review_count=('rating', 'count'),
    positive_sentiment=('sentiment', lambda x: (x.str.contains("POS", case=False, na=False)).sum()),
    negative_sentiment=('sentiment', lambda x: (x.str.contains("NEG", case=False, na=False)).sum()),
    feature_requests=('is_feature', 'sum'),
    bug_reports=('is_bug', 'sum')
).reset_index()

# Visualización de la evolución semanal
plt.figure(figsize=(12, 8))

# Subplot 1: Promedio de Rating por App
plt.subplot(2, 2, 1)
sns.lineplot(data=weekly_summary, x='week', y='avg_rating', hue='app')
plt.title('Promedio de Rating por App (semanal)')
plt.xlabel('Semana')
plt.ylabel('Promedio de Rating')

# Subplot 2: Cantidad de Reviews por App
plt.subplot(2, 2, 2)
sns.lineplot(data=weekly_summary, x='week', y='review_count', hue='app')
plt.title('Cantidad de Reviews por App (semanal)')
plt.xlabel('Semana')
plt.ylabel('Cantidad de Reviews')

# Subplot 3: Sentimientos positivos por App
plt.subplot(2, 2, 3)
sns.lineplot(data=weekly_summary, x='week', y='positive_sentiment', hue='app')
plt.title('Sentimientos Positivos por App (semanal)')
plt.xlabel('Semana')
plt.ylabel('Cantidad de Reviews')

# Subplot 4: Sentimientos negativos por App
plt.subplot(2, 2, 4)
sns.lineplot(data=weekly_summary, x='week', y='negative_sentiment', hue='app')
plt.title('Sentimientos Negativos por App (semanal)')
plt.xlabel('Semana')
plt.ylabel('Cantidad de Reviews')

plt.tight_layout()
plt.show()

## Guardar resultados

In [ ]:
# Guardar todas las reviews con análisis de sentimiento
all_reviews.to_csv("all_reviews.csv", index=False)

# Guardar el resumen semanal
weekly_summary.to_csv("weekly_summary.csv", index=False)

# Guardar feature requests
features_reviews.to_csv("features_reviews.csv", index=False)

# Guardar bug reports
bugs_reviews.to_csv("bugs_reviews.csv", index=False)